In [2]:
import os
os.chdir('/Users/zoe/Documents/复旦课程/大数据商业分析/pj2')
import json
import gensim
from gensim.models import Word2Vec
import random

Using TensorFlow backend.


In [7]:
# 得到歌单
with open('playlists.json', 'r', encoding='utf8') as file:
    contents0 = [json.loads(one) for one in file.readlines()]
print(len(contents0))

119143


In [3]:
with open('playlists(1).json', 'r', encoding='utf8') as file:
    contents = [json.loads(one) for one in file.readlines()]
print(len(contents))

143930


In [10]:
contents[120000:120005]

[{'createTime': 1513149161705,
  'description': '电子和女声擦除的火花\n\n只和品味相同的你分享',
  'expertTags': None,
  'experts': None,
  'highQuality': False,
  'id': 2013139534,
  'playCount': 1724,
  'shareCount': 3,
  'signature': '一生漂泊摆渡，临岸却是孤独。',
  'songs': [457552211],
  'tags': ['清新', '性感', '兴奋'],
  'trackCount': 1,
  'u_description': '',
  'u_detail': '',
  'updateTime': 1526354750683,
  'userId': 290932780,
  'userType': 0,
  'username': '李半知'},
 {'createTime': 1461224938746,
  'description': '歌单里都是评论两位数、非常冷门的日系女毒，每一首都是我的挚爱，希望也能惊艳到你呢！\n（封面原图见我动态）。\n[ 霓虹女毒 ] 冷门良曲，柔致心间 \n——挚爱女毒的alongClap',
  'expertTags': ['ACG', '日语'],
  'experts': None,
  'highQuality': False,
  'id': 367268443,
  'playCount': 112310,
  'shareCount': 71,
  'signature': '曾在网易云最好的时候，遇到过最好的一些人，度过了很开心的一段时光。回归初心，好好欣赏音乐本来的美。 曾经，网易云音乐 听见好时光',
  'songs': [30398710],
  'tags': ['日语', '治愈', '安静'],
  'trackCount': 1,
  'u_description': '',
  'u_detail': '',
  'updateTime': 1463508358109,
  'userId': 57192301,
  'userType': 0,
  'username'

In [6]:
# 歌单中歌曲数大于1的只有2010；
len(list(filter(lambda x:True if len(x['songs'])>1 else False, contents)))

2861

In [ ]:
filtered_contents = list(filter(lambda x:True if len(x['songs'])>1 else False, contents))

In [148]:
# 处理songs
songs = []
with open('songs.json', 'r', encoding='utf8') as file:
    for one in file.readlines():
        new_dict = {}
        one_dict = json.loads(one)
        new_dict['id'] = one_dict['id']
        new_dict['album'] = one_dict['album']
        new_dict['company'] = one_dict['album_company']
        new_dict['artist'] = one_dict['artist']
        new_dict['name'] = one_dict['name']
        new_dict['popularity'] = one_dict['popularity']
        songs.append(new_dict)
    
song_dict = {}
for one in songs:
    song_dict[one['id']] = one

In [ ]:
for one in filtered_contents[:100]:
    print(list(map(lambda x:song_dict[x]['name'], one['songs'])))

In [104]:
# 遍历所有歌单，生成模型
playlist_sequence = []
for one in filtered_contents:
    song_sequence = list(map(lambda x:str(x), one['songs']))
    for _ in range(len(song_sequence)):
        new = list(song_sequence)
        random.shuffle(new)
        playlist_sequence.append(new)

# out 为输出模型
out = 'song2vec.model'
model = Word2Vec(sentences=playlist_sequence, size=100, window=5, min_count=1)
model.save(out)

In [150]:
# 测试模型
def test(song_id):
    model = gensim.models.Word2Vec.load("song2vec.model")
    result = model.most_similar(str(song_id))
    print(song_dict[song_id])
    for one in result:
        print(((song_dict[int(one[0])]['artist'],song_dict[int(one[0])]['name'],song_dict[int(one[0])]['popularity']), one[1]))

In [152]:
for one in songs:
    if one['name'] == '齐天':
        print(one)

{'id': 481859794, 'album': '齐天', 'company': '奔跑怪物', 'artist': '华晨宇', 'name': '齐天', 'popularity': 100.0}
{'id': 32408691, 'album': '热门华语264', 'company': '', 'artist': '兔裹煎蛋卷', 'name': '齐天', 'popularity': 100.0}
{'id': 451104927, 'album': '洛天依', 'company': None, 'artist': '迁梦', 'name': '齐天', 'popularity': 95.0}


In [153]:
test(481859794)

{'id': 481859794, 'album': '齐天', 'company': '奔跑怪物', 'artist': '华晨宇', 'name': '齐天', 'popularity': 100.0}
(('华晨宇', '微光', 100.0), 0.9784262180328369)
(('华晨宇', '造物者', 100.0), 0.9724021553993225)
(('华晨宇', 'To Be Free', 100.0), 0.970414400100708)
(('华晨宇', '差不多先生 (Live) - live', 100.0), 0.9691726565361023)
(('华晨宇', '我管你', 100.0), 0.9668713808059692)
(('华晨宇', '卡西莫多的礼物', 100.0), 0.9658896327018738)
(('华晨宇', '故乡的云', 100.0), 0.962181568145752)
(('华晨宇', '趁你还年轻', 100.0), 0.9613037109375)
(('华晨宇', '春(Live版)', 100.0), 0.9612929821014404)
(('华晨宇', '环游', 100.0), 0.9603214263916016)
